In [1]:
%matplotlib inline
import os
import h5py
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils import data
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils import data

In [2]:
from scripts.dataloaders import dataLoaderDenoising

    
dataset = "BRATSmini5_train.h5"
# Getting the patient numbers
iDlist = []
with h5py.File(dataset, 'r' ) as f:
    keys = list(f.keys())
    for k in keys:
        # First split is patient iD
        iD = k.split("_")[0]
        if iD in iDlist:
            pass
        else:
            iDlist.append(iD)

Npat = len(iDlist)
trai_list = iDlist[0:Npat//3]
vali_list = iDlist[Npat//3:2*Npat//3]
test_list = iDlist[2*Npat//3:-1]


trai_data = dataLoaderDenoising(dataset, trai_list)
vali_data = dataLoaderDenoising(dataset, vali_list)
test_data = dataLoaderDenoising(dataset, test_list)

In [3]:
# U-Net parameters
wf = 8
depth = 4
in_chans= 1

# optimization
bs = 8
ilr = 3.5e-4
epo = 300

In [4]:

def loss_trai(cnn,x, y):
    return( F.l1_loss(y, cnn(x, thr=1)[0]) )

def loss_vali(cnn,x, y):
    return( F.l1_loss(y, cnn(x, thr=1)[0]) )

In [5]:
from scripts.training import trainingLoop

from CNNs.SUM import SUM_2d
cnn = SUM_2d(in_chans, depth, wf).float().cuda()

##############
# Epoch loop #
trainingLoop(
    cnn,
    trai_data,
    vali_data,
    epo, bs,
    loss_trai,
    loss_vali,
    torch.optim.AdamW(cnn.parameters(), lr=ilr, weight_decay=0.25),
    ilr,
    "./CNN_weights/SUM_DENO.pyt",
    printLog=True,
    logName="./CNN_weights/logs/SUM_DENO.log")

Epoch: 0 ,LR: 0.00035, Train Loss: 0.01225, Val Loss: 0.01217, Saved: 1
Epoch: 1 ,LR: 0.00035, Train Loss: 0.01069, Val Loss: 0.01054, Saved: 1
Epoch: 2 ,LR: 0.00035, Train Loss: 0.01042, Val Loss: 0.01027, Saved: 1
Epoch: 3 ,LR: 0.00035, Train Loss: 0.00978, Val Loss: 0.00959, Saved: 1
Epoch: 4 ,LR: 0.00035, Train Loss: 0.00997, Val Loss: 0.00980, Saved: 1
Epoch: 5 ,LR: 0.00035, Train Loss: 0.00946, Val Loss: 0.00926, Saved: 1
Epoch: 6 ,LR: 0.00034, Train Loss: 0.00959, Val Loss: 0.00938, Saved: 1
Epoch: 7 ,LR: 0.00034, Train Loss: 0.00950, Val Loss: 0.00931, Saved: 1
Epoch: 8 ,LR: 0.00034, Train Loss: 0.00924, Val Loss: 0.00902, Saved: 1
Epoch: 9 ,LR: 0.00034, Train Loss: 0.00923, Val Loss: 0.00902, Saved: 1
Epoch: 10 ,LR: 0.00034, Train Loss: 0.00925, Val Loss: 0.00904, Saved: 1
Epoch: 11 ,LR: 0.00034, Train Loss: 0.00908, Val Loss: 0.00887, Saved: 1
Epoch: 12 ,LR: 0.00034, Train Loss: 0.00920, Val Loss: 0.00900, Saved: 1
Epoch: 13 ,LR: 0.00034, Train Loss: 0.00902, Val Loss: 0.0087

Epoch: 113 ,LR: 0.00022, Train Loss: 0.00858, Val Loss: 0.00837, Saved: 1
Epoch: 114 ,LR: 0.00022, Train Loss: 0.00873, Val Loss: 0.00855, Saved: 1
Epoch: 115 ,LR: 0.00022, Train Loss: 0.00858, Val Loss: 0.00837, Saved: 1
Epoch: 116 ,LR: 0.00022, Train Loss: 0.00854, Val Loss: 0.00833, Saved: 1
Epoch: 117 ,LR: 0.00021, Train Loss: 0.00867, Val Loss: 0.00847, Saved: 1
Epoch: 118 ,LR: 0.00021, Train Loss: 0.00853, Val Loss: 0.00833, Saved: 1
Epoch: 119 ,LR: 0.00021, Train Loss: 0.00860, Val Loss: 0.00841, Saved: 1
Epoch: 120 ,LR: 0.00021, Train Loss: 0.00854, Val Loss: 0.00833, Saved: 1
Epoch: 121 ,LR: 0.00021, Train Loss: 0.00854, Val Loss: 0.00835, Saved: 1
Epoch: 122 ,LR: 0.00021, Train Loss: 0.00855, Val Loss: 0.00835, Saved: 1
Epoch: 123 ,LR: 0.00021, Train Loss: 0.00853, Val Loss: 0.00832, Saved: 1
Epoch: 124 ,LR: 0.00021, Train Loss: 0.00860, Val Loss: 0.00839, Saved: 1
Epoch: 125 ,LR: 0.00021, Train Loss: 0.00855, Val Loss: 0.00835, Saved: 1
Epoch: 126 ,LR: 0.00020, Train Loss: 0

Epoch: 224 ,LR: 0.00009, Train Loss: 0.00843, Val Loss: 0.00824, Saved: 1
Epoch: 225 ,LR: 0.00009, Train Loss: 0.00845, Val Loss: 0.00827, Saved: 1
Epoch: 226 ,LR: 0.00009, Train Loss: 0.00852, Val Loss: 0.00834, Saved: 1
Epoch: 227 ,LR: 0.00009, Train Loss: 0.00841, Val Loss: 0.00823, Saved: 1
Epoch: 228 ,LR: 0.00009, Train Loss: 0.00842, Val Loss: 0.00824, Saved: 1
Epoch: 229 ,LR: 0.00008, Train Loss: 0.00849, Val Loss: 0.00831, Saved: 1
Epoch: 230 ,LR: 0.00008, Train Loss: 0.00845, Val Loss: 0.00826, Saved: 1
Epoch: 231 ,LR: 0.00008, Train Loss: 0.00845, Val Loss: 0.00825, Saved: 1
Epoch: 232 ,LR: 0.00008, Train Loss: 0.00844, Val Loss: 0.00824, Saved: 1
Epoch: 233 ,LR: 0.00008, Train Loss: 0.00845, Val Loss: 0.00826, Saved: 1
Epoch: 234 ,LR: 0.00008, Train Loss: 0.00843, Val Loss: 0.00825, Saved: 1
Epoch: 235 ,LR: 0.00008, Train Loss: 0.00843, Val Loss: 0.00824, Saved: 1
Epoch: 236 ,LR: 0.00008, Train Loss: 0.00842, Val Loss: 0.00823, Saved: 1
Epoch: 237 ,LR: 0.00007, Train Loss: 0